# Download Waveform Accelerometer
Working Group Member 3 BMKG

***Authors:***
1.  Iman Fatchurochman, S.Si, M.DM
2.  Nova Heryandoko, S.Si, M.Si
3.  Ajat Sudrajat, M.Sc
4.  Bayu Pranata, M.Si
5.  Arif Nurokhim
6.  Kian Purna Sinki, SST, M.DM
7.  Melky Adi Kurniawan, S.Si
8.  Robby Wallansha, S.Tr
9.  Abdul Rosid, S.Tr, M.DM
10. Muhammad Fahmi Nugraha, S.Tr
11. Sesar Prabu Dwi Sriyanto
12. Ade Andika Saputra, S.Tr.Geof.
13. Rosi Budi Kurniawan, S.Tr.Geof.
14. Angga Wijaya, S.Tr.Geof.

In [ ]:
# Import module
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import sys, os
from obspy.geodetics import locations2degrees
from obspy.geodetics import degrees2kilometers
from obspy.signal.trigger import recursive_sta_lta
from obspy.taup import TauPyModel
import numpy as np
model = TauPyModel(model="iasp91")

In [ ]:
OT = UTCDateTime(sys.argv[1]) # 2012-04-11T08:38:34
longe = float(sys.argv[2])    # 93.03
late = float(sys.argv[3])     # 2.31
mag = float(sys.argv[4])      # 8.1
depth = float(sys.argv[5])    # 10
dirf = sys.argv[6]            # 20120411T083834_9303_0231_10_81

# Stasiun Accelerometer di Indonesia
file = open('stasiun.txt','r')
sta = file.readlines()
file.close()

for i in range(len(sta)):
    sta[i] = sta[i].split()

# Membuat directory
if not os.path.isdir(dirf):
    os.mkdir(dirf)
if not os.path.isdir(dirf+'/sta_trig'):
    os.mkdir(dirf+'/sta_trig')
if not os.path.isdir(dirf+'/sta_no_trig'):
    os.mkdir(dirf+'/sta_no_trig')

# Server FDSN Accelerometer
client = Client("http://172.19.3.212:8080/")

# Logging stasiun yang digunakan
file = open(dirf+'/sta_data.txt','w')

# Download waveform masing-masing stasiun
for s in sta:
    dist = locations2degrees(late, longe, float(s[1]), float(s[0]))
    if dist > 15:
        file.write(s[0]+' '+s[1]+' '+s[2]+' 3\n')
        continue
    arrivals = model.get_travel_times(source_depth_in_km=depth, distance_in_degree=dist, phase_list=["P", "p", "S", "s"])
    t0 = OT + arrivals[0].time - 20
    t1 = OT + arrivals[-1].time + 3*60
    dist = str(round(degrees2kilometers(dist),2))
    
    try:
        st = client.get_waveforms("IA", s[2], "*", "HN?", t0, t1)
    except:
        try:
            st = client.get_waveforms("IA", s[2], "*", "SL?", t0, t1)
        except:
            print(s[2], t0, t1, 'No Data')
            file.write(s[0]+' '+s[1]+' '+s[2]+' '+dist+' 0\n')
            continue
            
    gaps = st.get_gaps(min_gap=1)
    if gaps:
        print(s, 'Data gap')
        file.write(s[0]+' '+s[1]+' '+s[2]+' '+dist+' 2\n')
        continue
        
    st.merge(method=1,fill_value="interpolate")
    if st.count() < 3:
        print(s, 'Not full')
        file.write(s[0]+' '+s[1]+' '+s[2]+' '+dist+' 2\n')
        continue
        
    tr = st.select(channel='??Z')[0]
    cft = recursive_sta_lta(tr.data, int(5 * tr.stats.sampling_rate), int(10 * tr.stats.sampling_rate))
    on_of = np.where(cft > 1.5)[0]
    
    if len(on_of)>0:
        st.write(dirf+'/sta_trig/'+s[2]+'.mseed')
        st.plot(equal_scale='False', outfile=dirf+"/sta_trig/waveform_"+s[2]+".png")
    else:
        st.write(dirf+'/sta_no_trig/'+s[2]+'.mseed')
        st.plot(equal_scale='False', outfile=dirf+"/sta_no_trig/waveform_"+s[2]+".png")
    file.write(s[0]+' '+s[1]+' '+s[2]+' '+dist+' 1\n')
file.close()